In [ ]:
#!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-q4.gguf --local-dir . --local-dir-use-symlinks False
#!cp ./Phi-3-mini-4k-instruct-q4.gguf /content/drive/MyDrive/
#!pip install --upgrade pip setuptools wheel
#!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip wheel llama-cpp-python -w /content/drive/MyDrive/pip_cache

  Using cached llama_cpp_python-0.2.78.tar.gz (50.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 57.0 MB/s eta 0:00:00
Saved ./drive/MyDrive/pip_cache/diskcache-5.6.3-py3-none-any.whl
Saved ./drive/MyDrive/pip_cache/jinja2-3.1.4-py3-none-any.whl
Saved ./drive/MyDrive/pip_cache/numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Saved ./drive/MyDrive/pip_cache/typing_extensions-4.12.2-py3-none-any.whl
Saved ./drive/MyDrive/pip_cache/MarkupSafe-2.1.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Created wheel for llama-cpp-python

In [1]:
!pip install /content/drive/MyDrive/pip_cache/llama_cpp_python-*.whl
from llama_cpp import Llama
import pandas as pd

Processing ./drive/MyDrive/pip_cache/llama_cpp_python-0.2.78-cp310-cp310-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.3 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train_synthetic.csv')

In [2]:
llm = Llama(
  model_path="/content/drive/MyDrive/Phi-3-mini-4k-instruct-q4.gguf",
  n_ctx=4096,
  n_threads=8,
  n_gpu_layers=35
)


llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /content/drive/MyDrive/Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_

In [19]:
index = 2
user = df.iloc[index]['claude_incomplete'].replace('USER:','<|user|>\n')
system = df.iloc[index]['system'].replace('SYSTEM:','<|system|>\n')

prompt = f'''{system}<|end|>
          You can perform following actions:
          1. If some parameters are missing ask the user for the rest of required parameters.
          2. Only when the user provided all of the required parameters return the API call in the format:
          <function_call>function_name{{parameters}}</function_call> and nothing else!
          {user}<|end|>\n<|assistant|>'''

# Simple inference example
output = llm(
  prompt,
  max_tokens=256,  # Generate up to 256 tokens
  stop=["<|end|>"],
  echo=False,  # Whether to echo the prompt
)
#print(f"<|user|>\n{prompt}<|end|>\n<|assistant|>")
print(prompt)
print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     579.48 ms
llama_print_timings:      sample time =      64.91 ms /   109 runs   (    0.60 ms per token,  1679.30 tokens per second)
llama_print_timings: prompt eval time =     371.84 ms /   437 tokens (    0.85 ms per token,  1175.25 tokens per second)
llama_print_timings:        eval time =    1935.28 ms /   108 runs   (   17.92 ms per token,    55.81 tokens per second)
llama_print_timings:       total time =    2431.10 ms /   545 tokens


<|system|>
 You are a helpful assistant with access to the following functions. Use them if required -
{
    "name": "generate_password",
    "description": "Generate a random password",
    "parameters": {
        "type": "object",
        "properties": {
            "length": {
                "type": "integer",
                "description": "The length of the password"
            },
            "include_symbols": {
                "type": "boolean",
                "description": "Whether to include symbols in the password"
            }
        },
        "required": [
            "length"
        ]
    }
}

{
    "name": "create_task",
    "description": "Create a new task in a task management system",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "The title of the task"
            },
            "due_date": {
                "type": "string",
                "format"